# Exploring different feature ranges to assess their impact on our models' performance. 

In [1]:
# increase the width of the notebook
from IPython.display import display, HTML, Markdown

display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [3]:
# Load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

y_train = train["Score"]
y_test = test["Score"]

X_train = train.drop("Score", axis=1)
X_test = test.drop("Score", axis=1)

# Elo Ranges

In [4]:
# Define masks for the three Elo-based groups
low_mask   =  X_test["WhiteElo"]   < 1000
mid_mask   = (X_test["WhiteElo"]  >= 1000) & (X_test["WhiteElo"] < 2000)
high_mask  =  X_test["WhiteElo"]   >= 2000

# Slice X and y for each group
X_test_low,   y_test_low   = X_test[low_mask],   y_test[low_mask]
X_test_mid,   y_test_mid   = X_test[mid_mask],   y_test[mid_mask]
X_test_high,  y_test_high  = X_test[high_mask],  y_test[high_mask]

In [5]:
#Preprocessing pipelines
numeric_features = ["WhiteElo", "EloDif"]
categorical_features = ["Opening_name", "Time_format", "Increment_binary"]

numeric_transformer = Pipeline([
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

In [6]:
preprocessor.fit(X_train)
X_test_low = preprocessor.transform(X_test_low)
X_test_mid = preprocessor.transform(X_test_mid)
X_test_high = preprocessor.transform(X_test_high)

In [7]:
ada_boost = joblib.load("Models/best_adaboost.joblib")
random_forest = joblib.load("Models/best_random_forest.joblib")
decision_tree = joblib.load("Models/best_decision_tree.joblib")
#decision_tree_pca = joblib.load("Models/best_decision_tree_pca.joblib")
best_gradient_boosting = joblib.load("Models/best_gradient_boosting.joblib")
best_svc_poly = joblib.load("Models/best_svc_poly.joblib")
best_svc_rbf = joblib.load("Models/best_svc_rbf.joblib")
voting_clf = joblib.load("Models/voting_clf.joblib")

In [8]:
models = [ada_boost, random_forest, best_gradient_boosting,decision_tree, best_svc_poly, best_svc_rbf, voting_clf]
model_names = ["AdaBoost", "Random_forest","Gradient Boosting","Decision Tree", "SVC (poly)", "SVC (rbf)", "voting_clf"]

In [9]:
for grp_name, X_sub, y_sub in [
    ("Low (<1000)",    X_test_low,  y_test_low),
    ("Mid (1000–1999)",X_test_mid,  y_test_mid),
    ("High (>=2000)",  X_test_high, y_test_high),
]:
    
    print(f"\n=== {grp_name} group ===")
    print("n_samples:", len(y_sub))
    for model, name in zip(models, model_names):
        y_pred = model.predict(X_sub)
        acc    = accuracy_score(y_sub, y_pred)
        print(f"  {name:<20} Accuracy: {acc:.3f}")
        # print(classification_report(y_sub, y_pred))
        



=== Low (<1000) group ===
n_samples: 167
  AdaBoost             Accuracy: 0.581
  Random_forest        Accuracy: 0.545
  Gradient Boosting    Accuracy: 0.557
  Decision Tree        Accuracy: 0.497
  SVC (poly)           Accuracy: 0.479
  SVC (rbf)            Accuracy: 0.551
  voting_clf           Accuracy: 0.545

=== Mid (1000–1999) group ===
n_samples: 3583
  AdaBoost             Accuracy: 0.530
  Random_forest        Accuracy: 0.541
  Gradient Boosting    Accuracy: 0.524
  Decision Tree        Accuracy: 0.507
  SVC (poly)           Accuracy: 0.519
  SVC (rbf)            Accuracy: 0.529
  voting_clf           Accuracy: 0.523

=== High (>=2000) group ===
n_samples: 6250
  AdaBoost             Accuracy: 0.556
  Random_forest        Accuracy: 0.552
  Gradient Boosting    Accuracy: 0.554
  Decision Tree        Accuracy: 0.539
  SVC (poly)           Accuracy: 0.544
  SVC (rbf)            Accuracy: 0.549
  voting_clf           Accuracy: 0.552


### Our analysis of different Elo rating groups reveals the best model performance within the under 1000 Elo group. However, with a sample size of only 167, this finding might not be statistically robust. Among the other two groups (1000-1999 and >= 2000 Elo), the group with an Elo rating of 2000 or higher shows approximately 2% better prediction accuracy compared to the 1000-1999 Elo group.

## Elo difference

In [10]:
# Build masks
mask_diff_low  = X_test["EloDif"].abs() < 100
mask_diff_mid  = (X_test["EloDif"].abs() >= 100) & (X_test["EloDif"].abs() < 200)
mask_diff_high = X_test["EloDif"].abs() >= 200

#  Slice the raw test-set X **and** y
X_diff_low,  y_diff_low  = X_test[mask_diff_low],  y_test[mask_diff_low]
X_diff_mid,  y_diff_mid  = X_test[mask_diff_mid],  y_test[mask_diff_mid]
X_diff_high, y_diff_high = X_test[mask_diff_high], y_test[mask_diff_high]

# Transform each feature‑subset
X_diff_low  = preprocessor.transform(X_diff_low)
X_diff_mid  = preprocessor.transform(X_diff_mid)
X_diff_high = preprocessor.transform(X_diff_high)

#  Evaluate
for grp_name, X_sub, y_sub in [
    ("Elo_Diff_Low (<100)",    X_diff_low,  y_diff_low),
    ("Elo_Diff_Mid (100–200)", X_diff_mid,  y_diff_mid),
    ("Elo_Diff_High (>=200)",  X_diff_high, y_diff_high),
]:
    print(f"\n=== {grp_name} group ===")
    print("n_samples:", len(y_sub))
    for model, name in zip(models, model_names):
        y_pred = model.predict(X_sub)
        acc    = accuracy_score(y_sub, y_pred)
        print(f"  {name:<20} Accuracy: {acc:.3f}")



=== Elo_Diff_Low (<100) group ===
n_samples: 7654
  AdaBoost             Accuracy: 0.525
  Random_forest        Accuracy: 0.526
  Gradient Boosting    Accuracy: 0.521
  Decision Tree        Accuracy: 0.499
  SVC (poly)           Accuracy: 0.510
  SVC (rbf)            Accuracy: 0.519
  voting_clf           Accuracy: 0.518

=== Elo_Diff_Mid (100–200) group ===
n_samples: 1800
  AdaBoost             Accuracy: 0.590
  Random_forest        Accuracy: 0.590
  Gradient Boosting    Accuracy: 0.590
  Decision Tree        Accuracy: 0.588
  SVC (poly)           Accuracy: 0.584
  SVC (rbf)            Accuracy: 0.590
  voting_clf           Accuracy: 0.590

=== Elo_Diff_High (>=200) group ===
n_samples: 546
  AdaBoost             Accuracy: 0.711
  Random_forest        Accuracy: 0.712
  Gradient Boosting    Accuracy: 0.711
  Decision Tree        Accuracy: 0.711
  SVC (poly)           Accuracy: 0.705
  SVC (rbf)            Accuracy: 0.711
  voting_clf           Accuracy: 0.711


### The group with an Elo difference less than the absolute value of 100 (|Elo_diff| < 100) has the lowest accuracy and the largest sample size. The Elo_Diff_Mid (100–200) group shows improved accuracy, reaching 59% with a decent number of samples (1800). The Elo_Diff_High (>=200) group achieves an impressive 71% accuracy, strongly indicating that a large difference in Elo ratings leads to highly predictable outcomes.

# Time Formats

In [11]:
#  Build the mask on the raw test set
mask_bullet = X_test["Time_format"].str.strip() == "bullet"
mask_blitz = X_test["Time_format"].str.strip() == "blitz"
mask_rapid = X_test["Time_format"].str.strip() == "rapid"
mask_classical = X_test["Time_format"].str.strip() == "classical"


# Slice X and y for each group
X_test_bullet,   y_test_bullet   = X_test[mask_bullet],   y_test[mask_bullet]
X_test_blitz,  y_test_blitz  = X_test[mask_blitz],  y_test[mask_blitz]
X_test_rapid,  y_test_rapid  = X_test[mask_rapid],  y_test[mask_rapid]
X_test_classical,  y_test_classical  = X_test[mask_classical],  y_test[mask_classical]

In [12]:
# Transform each feature‑subset
X_test_bullet  = preprocessor.transform(X_test_bullet)
X_test_blitz  = preprocessor.transform(X_test_blitz)
X_test_rapid = preprocessor.transform(X_test_rapid)
X_test_classical = preprocessor.transform(X_test_classical)

In [13]:
#  Evaluate
for grp_name, X_sub, y_sub in [
    ("bullet",    X_test_bullet,  y_test_bullet),
    ("blitz", X_test_blitz,  y_test_blitz),
    ("rapid",  X_test_rapid, y_test_rapid),
    ("classical",  X_test_classical, y_test_classical),
]:
    print(f"\n=== {grp_name} group ===")
    print("n_samples:", len(y_sub))
    for model, name in zip(models, model_names):
        y_pred = model.predict(X_sub)
        acc    = accuracy_score(y_sub, y_pred)
        print(f"  {name:<20} Accuracy: {acc:.3f}")


=== bullet group ===
n_samples: 3972
  AdaBoost             Accuracy: 0.538
  Random_forest        Accuracy: 0.550
  Gradient Boosting    Accuracy: 0.534
  Decision Tree        Accuracy: 0.525
  SVC (poly)           Accuracy: 0.537
  SVC (rbf)            Accuracy: 0.544
  voting_clf           Accuracy: 0.540

=== blitz group ===
n_samples: 3587
  AdaBoost             Accuracy: 0.545
  Random_forest        Accuracy: 0.539
  Gradient Boosting    Accuracy: 0.544
  Decision Tree        Accuracy: 0.522
  SVC (poly)           Accuracy: 0.525
  SVC (rbf)            Accuracy: 0.534
  voting_clf           Accuracy: 0.536

=== rapid group ===
n_samples: 2015
  AdaBoost             Accuracy: 0.567
  Random_forest        Accuracy: 0.556
  Gradient Boosting    Accuracy: 0.562
  Decision Tree        Accuracy: 0.545
  SVC (poly)           Accuracy: 0.549
  SVC (rbf)            Accuracy: 0.556
  voting_clf           Accuracy: 0.558

=== classical group ===
n_samples: 426
  AdaBoost             Accura